In [ ]:
import spacy
import random
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount("my-drive")

Mounted at my-drive


In [ ]:
df=pd.read_excel("my-drive/My Drive/annotations1.xlsx")
df1=pd.read_excel("my-drive/My Drive/facultyinfo1.xlsx")

In [ ]:
ids={"e_2":"Name","e_3":"Specialization","e_4":"Address","e_5":"Department","e_6":"Designation","e_7":"Degree","e_8":"Email","e_9":"Experience","e_10":"MobileNumber","e_13":"PersonalWebsite","e_14":"Qualification"}

training_data=[]

In [ ]:
def prepareTrainingData(df,df1,ids,training_data):
  for i in range(len(df1.index)):
    temp_dict={}
    temp_dict["entities"]=[]
    temp_list=[]
    temp_list.append(df1.iloc[i,0])
    data=json.loads(df.iloc[i,0])
    list1=data["entities"]
    
    for element in list1:
        ann_text=ids.get(element['classId'])
        if(ann_text==None):
            continue
        start=element['offsets'][0]['start']
        end=element['offsets'][0]['start']+len(element['offsets'][0]['text'])
        l1=[]
        l1.append(start)
        l1.append(end)
        l1.append(ann_text)
        l1=tuple(l1)
        temp_dict["entities"].append(l1)
    
    temp_list.append(temp_dict)
    training_data.append(tuple(temp_list))
  return training_data
# prepareTrainingData(df,df1,ids,training_data)

In [ ]:
training_data=prepareTrainingData(df,df1,ids,training_data)

In [ ]:
stdf=pd.read_excel('my-drive/My Drive/stanford annotation.xlsx')
stdf1=pd.read_excel('my-drive/My Drive/stanfordfacultyinfo.xlsx')

In [ ]:
ids1={"e_8":"Experience","e_3":"Address","e_2":"Specialization","e_7":"Email","e_5":"Designation","e_11":"Qualification","e_6":"Degree","e_9":"MobileNumber","e_10":"PersonalWebsite","e_4":"Department","e_1":"Name"}

In [ ]:
training_data1=[]
training_data1=prepareTrainingData(stdf,stdf1,ids1,training_data1)

In [ ]:
training_data.extend(training_data1)

In [ ]:
mitdf=pd.read_excel('my-drive/My Drive/mit annotation.xlsx')
mitdf1=pd.read_csv('my-drive/My Drive/mitfacultyinfo.csv')

In [ ]:
training_data2=[]
trainig_data2=prepareTrainingData(mitdf,mitdf1[:19],ids1,training_data2)

In [ ]:
training_data2

[('Polina  Anikeeva,Associate Professor in Materials Science and Engineering,Associate Professor in Brain and Cognitive Sciences,McGovern Institute for Brain Research,Associate Director, Research Laboratory of Electronics,B.S. in Biophysics, St. Petersburg State Polytechnic University, 2003,Ph.D. in Materials Science and Engineering, MIT, 2009,Room 36-849,anikeeva@mit.edu,Bioelectronics Group,CV,Bio; Biomaterials; Biophysics; Biotechnology; Electronic Materials; Magnetic Materials; Materials Chemistry; Implants; Nanotechnology; Photonic Materials,Research,Prof. Polina Anikeeva joined DMSE in 2011. Dr. Anikeeva\'s Ph.D. thesis focussed on physical properties and design of light emitting devices based on organic materials and nanoparticles, working under the supervision of Prof. Vladimir Bulovic in EECS. She previously held the Dean\'s Postdoctoral Fellowship, School of Medicine, Stanford and was in the group of Prof. Karl Deisseroth in the Department of Bioengineering. Her current resea

In [ ]:
len(training_data)

68

In [ ]:
for tp in training_data:
  dic=tp[1]
  prevend=-1
  for tp1 in dic['entities']:
    start=tp1[0]
    end=tp1[1]
    if prevend>=start:
      print('Overlapping in '+tp[0][:10])
    prevend=end


In [ ]:
for tp in training_data2:
  dic=tp[1]
  prevend=-1
  for tp1 in dic['entities']:
    start=tp1[0]
    end=tp1[1]
    if prevend>=start:
      print('Overlapping in '+tp[0][:10])
    prevend=end

Overlapping in Jeffrey C.


In [ ]:
training_data.extend(training_data2)

In [ ]:
def train_model(data,iterations):
  TRAIN_DATA=data
  nlp=spacy.blank('en')

  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  
    optimizer = nlp.begin_training()

    for itn in range(iterations):
      random.shuffle(TRAIN_DATA)
      losses = {}

      for text, annotations in TRAIN_DATA:
        nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,  
                    losses=losses)
  return nlp

In [ ]:
nlp_model=train_model(training_data,25)

In [ ]:
nlp_model=spacy.load('my-drive/My Drive/nlp_model')

In [ ]:
test_df=pd.read_csv('my-drive/My Drive/facultyinfo.csv')

In [ ]:
update_df=pd.read_excel('my-drive/My Drive/caltech.xlsx')

In [ ]:
update_df['Change']='No'

In [ ]:
ent_weights={'Specialization':10,'Address':30,'Experience':20,'Email':50,'MobileNumber':40,'Qualification':20,'Designation':20,'PersonalWebsite':25,'Department':20,'Degree':20}

In [ ]:
for i in range(len(test_df)):
  for j in range(len(update_df)):
    total_weight=0
    if str(test_df.at[i,'url']) in str(update_df.at[j,'URL']):
      print('Predicting')
      doc=nlp_model(str(test_df.at[i,'content']))
      for ent in doc.ents:
        if len(ent.text)<3:
          continue
        print(ent.text+'\t'+ent.label_)
        if ent.label_=='Specialization':
          if ent.text not in str(update_df.at[j,'Area of Specialization(in words)']):
            update_df.at[j,'Area of Specialization(in words)']=str(update_df.at[j,'Area of Specialization(in words)'])+', '+ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Address':
          if ent.text not in str(update_df.at[j,'Corresponding Address']):
            update_df.at[j,'Corresponding Address']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Experience':
          if ent.text not in str(update_df.at[j,'Experience']):
            update_df.at[j,'Experience']=str(update_df.at[j,'Experience'])+', '+ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Email':
          if ent.text not in str(update_df.at[j,'E-mail']):
            update_df.at[j,'E-mail']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='MobileNumber':
          if ent.text not in str(update_df.at[j,'Mobile No']):
            update_df.at[j,'Mobile No']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Qualification':
          if ent.text not in str(update_df.at[j,'Qualification']):
            update_df.at[j,'Qualification']=str(update_df.at[j,'Qualification'])+', '+ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Designation':
          if ent.text not in str(update_df.at[j,'Designation']):
            update_df.at[j,'Designation']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='PersonalWebsite':
          if ent.text not in str(update_df.at[j,'Personal Website']):
            update_df.at[j,'Personal Website']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Department':
          if ent.text not in str(update_df.at[j,'Department']):
            update_df.at[j,'Department']=ent.text
            total_weight+=ent_weights[ent.label_]
        elif ent.label_=='Degree':
          if ent.text not in str(update_df.at[j,'Doctorate Degree']):
            update_df.at[j,'Doctorate Degree']=ent.text
            total_weight+=ent_weights[ent.label_]
      print(total_weight)
      if total_weight>50:
        update_df.at[j,'Change']='Yes'
        print('Changed')

Predicting
P. P. Vaidyanathan	Name
Professor of	Designation
20
Predicting
Venkat Chandrasekaran	Name
Professor	Designation
Caltech in Computing and Mathematical Sciences and in Electrical Engineering	Qualification
Mathematics of Data Science	Specialization
mathematical	Specialization
interface with topics in the information sciences	Experience
60
Changed
Predicting
Sanjoy Mahajan	Name
Associate Professor	Designation
physics	Specialization
physics	Specialization
0
Predicting
Skip to	Name
Calvin Lab	Name
People,Overview,Scientific Leadership,Staff,Current Long-Term Visitors,Research Fellows,Scientific Advisory Board,Governance Board,Affiliated Faculty,Science Communicator in Residence,Programs & Events,Overview,Programs,Workshops & Symposia,Internal Program Activities,Public Lectures,Participate,Visiting,Overview,Directions,Berkeley & the Bay Area,Accommodation,Visas,Families,IT Guide,Bicycle Loans,Code of Conduct,Support,Overview,Simons Foundation,Industrial Partnership,Academic Partner

In [ ]:
update_df[update_df['Change']=='Yes']

,Academician Name,Area of Specialization(in words),Corresponding Address,Country Name (editable),Date of Birth,Date of Entry,Department,Department Name,Designation,Designation Detailed,Doctorate Degree,Doctorate Discipline,Doctorate Passing Year,Doctorate University Name,E-mail,Email Id,Employer Name(University/Research Lab Name),Experience,Gender,Graduation Degree,Graduation Discipline,Graduation Passing Year,Graduation University Name,Image URL,Indian University Name OR College Name,Major Area,Mentor Name,Minor Area,Mobile No,Nationality,Office Address,Permanent Tenure Track Contract,Personal Website,Post Graduation Degree,Post Graduation Discipline,Post Graduation Passing Year,Post Graduation University Name,Preferred Indian City,Qualification,Reason For Indian,Remarks,Specialization Keywords,Total Experience (in years),Twitter Id,Type,URL,Change
5,Srikanth Venkata Tenneti,"Fuses,Electric current ,ac and dc ,frequency g...",Mavrix Tech,United States of America,NaN,"Friday, September 30, 2016",Engineering,Electrical Engineering,Chief Scientist,Graduate Student,PhD,Digital Signal Processing,NaN,California Institute of Technology,stennetic@altech.edu,stennetic@altech.edu,California Institute of Technology,Graduate Student\nCalifornia Institute of Tech...,Male,BTech,Electrical Engineering,2012.0,"Indian Institute of Technology, Bombay",http://systems.caltech.edu/dsp/students/srikan...,Others,Electric Machine,NaN,Electric Motor,Not Found,Indian,MC 136-93\nPasadena CA 91125,NaN,Not Found,MS,Electrical Engineering,2014.0,California Institute of Technology,Bombay,California Institute of Technology\nDoctor of ...,Name+photo+education+birth,Not Found,"Electric Machine,Electric Motor,Induction Motor",NaN,Not Found,Phd Student,https://directory.caltech.edu/personnel/stenne...,Yes
10,Raj Katti,"Computational Biology and Biophysics,physics, ...",Not Found,United States of America,NaN,"Friday, September 30, 2016",Physical Science,Physics,Graduate Student,Graduate Student,Not Found,Not Found,NaN,Not Found,rmkatti@caltech.edu,rmkatti@caltech.edu,California Institute of Technology,"Not Found, Raj,Katti,Graduate Student,Caltech",Male,BS,Physics,2014.0,California Institute of Technology,https://media.licdn.com/mpr/mpr/shrinknp_400_4...,Others,Computational Biology,NaN,Biophysics,Not Found,Indian,MC 149-33\nPasadena CA 91125,NaN,Not Found,Not Found,Not Found,NaN,Not Found,Not Found,California Institute of Technology\nBachelor o...,Name+photo,Not Found,"Computational Biology, Biophysics",NaN,Not Found,Phd Student,https://www.google.co.in/search?q=Raj+M.+Katti...,Yes
23,Kartik Srinivasan,"nanophotonics, quantum optics, quantum dots, n...",Not Found,United States of America,NaN,"Friday, September 30, 2016",Computer Science,Physics,Caltech Library Service,PostDoctoral Scholar,PhD,Applied Physics,NaN,California Institute of Technology,kartik.srinivasan@nist.gov,kartik.srinivasan@nist.gov,California Institute of Technology,Not Found,Male,BS,Applied Physics,NaN,California Institute of Technology,https://www.nist.gov/sites/default/files/style...,Others,Quantum Optics,NaN,Quantum Dots,301 975 5938,Indian,"Department of Applied Physics, California Inst...",NaN,Not Found,MS,Applied Physics,NaN,California Institute of Technology,Chennai,"B.S., M.S., and Ph.D. degrees in Applied Physi...",Name+photo+education,Not Found,"Quantum Optics, Quantum Dots",NaN,Not Found,Faculty,"https://www.nist.gov/people/kartik-srinivasan,...",Yes
24,Shwetank Kumar,"nanophotonics, quantum optics, quantum dots, n...",Not Found,United States of America,NaN,"Friday, September 30, 2016",Computer Science,Physics,Caltech Submillimeter,PostDoctoral Scholar,PhD,Applied Physics,2007.0,California Institute of Technology,shwetank.kumar@gmail.com,shwetank.kumar@gmail.com,California Institute of Technology,"VP, Product Engineering, Orbital Insight Orbit...",Male,BE,Electrical Engineering,2000.0,IIT Delhi,https://media.licdn.com/mpr/mpr/shrinknp_400_4...,Others,Quantum Dots,NaN,Nanofabricatio

In [ ]:
update_df.to_csv('my-drive/My Drive/caltech_updated.csv',index=False)

In [ ]:
nlp_model.to_disk('my-drive/My Drive/nlp_model')

In [ ]:
newdf=pd.read_csv('my-drive/My Drive/newfacultyinfo2.csv')

In [ ]:
newdf_dict={"Name":[],"Experience":[],"Address":[],"Specialization":[],"Email":[],"Designation":[],"Qualification":[],"Degree":[],"MobileNumber":[],"PersonalWebsite":[],"Department":[],"URL":[]}

In [ ]:
newdf

,content,name,url
0,"Skip to content,Simons Institute for the Theor...",Prasad Raghavendra,https://simons.berkeley.edu/people/prasad-ragh...
1,"Skip to content,Simons Institute for the Theor...",Shankar Sastry,https://simons.berkeley.edu/people/shankar-sastry
2,"Skip to content,Simons Institute for the Theor...",Nikhil Srivastava,https://simons.berkeley.edu/people/nikhil-sriv...
3,"Skip to content,Simons Institute for the Theor...",Satish Rao,https://simons.berkeley.edu/people/satish-rao
4,"Skip to content,Simons Institute for the Theor...",Kameshwar Poolla,https://simons.berkeley.edu/people/kameshwar-p...
5,"Skip to content,Simons Institute for the Theor...",Ashvin Vishwanath,https://simons.berkeley.edu/people/ashvin-vish...
6,"Skip to content,Simons Institute for the Theor...",Barna Saha,https://simons.berkeley.edu/people/barna-saha
7,"Skip to content,Simons Institute for the Theor...",Gireeja Ranade,https://simons.berkeley.edu/people/gireeja-ranade
8,"Skip to content,Main navigation,Main content,U...",Ned Mohan,https://ece.umn.edu/directory/mohan-ned/
9,"Skip to content,Simons Institute for the Theor...",Venkat Anantharam,https://simons.berkeley.edu/people/venkat-anan...


In [ ]:
for i in range(len(newdf)):
  Experience='Not Found'
  Address='Not Found'
  Specialization='Not Found'
  Email='Not Found'
  Designation='Not Found'
  Qualification='Not Found'
  Degree='Not Found'
  MobileNumber='Not Found'
  PersonalWebsite='Not Found'
  Department='Not Found'
  doc=nlp_model(newdf.at[i,'content'])
  for ent in doc.ents:
    if ent.label_=='Experience':
      if Experience!='Not Found':
        Experience=Experience+", "+ent.text
      else:
        Experience=ent.text
    elif ent.label_=='Address':
      if Address=='Not Found':
        Address=ent.text
    elif ent.label_=='Specialization':
      if Specialization!='Not Found':
        Specialization=Specialization+", "+ent.text
      else:
        Specialization=ent.text
    elif ent.label_=='Email':
      if Email=='Not Found' and ('@' in ent.text):
        Email=ent.text
    elif ent.label_=='Designation':
      if Designation=='Not Found':
        Designation=ent.text
    elif ent.label_=='Qualification':
      if Qualification!='Not Found':
        Qualification=Qualification+", "+ent.text
      else:
        Qualification=ent.text
    elif ent.label_=='Degree':
      if Degree!='Not Found':
        Degree=Degree+", "+ent.text
      else:
        Degree=ent.text
    elif ent.label_=='MobileNumber':
      if MobileNumber=='Not Found':
        MobileNumber=ent.text
    elif ent.label_=='PersonalWebsite':
      if PersonalWebsite=='Not Found':
        PersonalWebsite=ent.text
    elif ent.label_=='Department':
      if Department=='Not Found':
        Department=ent.text
    print(ent.text+"\t"+ent.label_)
  newdf_dict['Address'].append(Address)
  newdf_dict['Degree'].append(Degree)
  newdf_dict['Department'].append(Department)
  newdf_dict['Designation'].append(Designation)
  newdf_dict['Email'].append(Email)
  newdf_dict['Experience'].append(Experience)
  newdf_dict['MobileNumber'].append(MobileNumber)
  newdf_dict['Name'].append(newdf.at[i,'name'])
  newdf_dict['PersonalWebsite'].append(PersonalWebsite)
  newdf_dict['Qualification'].append(Qualification)
  newdf_dict['Specialization'].append(Specialization)
  newdf_dict['URL'].append(newdf.at[i,'url'])

newdatadf=pd.DataFrame.from_dict(newdf_dict)

Skip to	Name
Calvin Lab	Name
People,Overview,Scientific Leadership,Staff,Current Long-Term Visitors,Research Fellows,Scientific Advisory Board,Governance Board,Affiliated Faculty,Science Communicator in Residence,Law & Society Fellow,Programs & Events,Overview,Programs,Workshops & Symposia,Internal Program Activities,Public Lectures,Participate,Visiting,Overview,Directions,Berkeley & the Bay Area,Accommodation,Visas,Families,IT Guide,Bicycle Loans,Code of Conduct,Support,Overview,Simons Foundation,Industrial Partnership,Academic Partners,Donate,Richard M. Karp Fund,Watch / Read,SimonsTV,Program Reports,News Stories,Calvin Café (blog),Calendar,You are here,Home,People,Current Long-Term Visitors,Prasad Raghavendra,Senior Scientist, Simons Institute, UC Berkeley,Website,Prasad Raghavendra is senior scientist	Email
associate professor	Designation
Quantum Wave	Specialization
Quantum Computation	Specialization
Program Organizer and Workshop Organizer.,Real Analysis in Computer Science	Experi

In [ ]:
newdatadf

,Name,Experience,Address,Specialization,Email,Designation,Qualification,Degree,MobileNumber,PersonalWebsite,Department,URL
0,Prasad Raghavendra,"Program Organizer and Workshop Organizer.,Real...",Not Found,"Quantum Wave, Quantum Computation",Not Found,associate professor,Not Found,Not Found,Not Found,Not Found,Not Found,https://simons.berkeley.edu/people/prasad-ragh...
1,Shankar Sastry,Not Found,Not Found,Not Found,Not Found,Siebel Professor,Not Found,Not Found,Not Found,Not Found,Computer Science,https://simons.berkeley.edu/people/shankar-sastry
2,Nikhil Srivastava,"researcher, Program Organizer and Workshop Org...",Not Found,"Mathematics, Quantum Computation",Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Computer Science,https://simons.berkeley.edu/people/nikhil-sriv...
3,Satish Rao,Not Found,Not Found,scientist,Not Found,professor,Not Found,Not Found,Not Found,Not Found,computer science,https://simons.berkeley.edu/people/satish-rao
4,Kameshwar Poolla,Not Found,Not Found,Not Found,Not Found,Professor in,"Indian Institute of Technology, Bombay in 1980",Not Found,Not Found,Not Found,Mechanical Engineering,https://simons.berkeley.edu/people/kameshwar-p...
5,Ashvin Vishwanath,Not Found,Not Found,Not Found,Not Found,Assistant Professor,Not Found,Not Found,Not Found,Not Found,Not Found,https://simons.berkeley.edu/people/ashvin-vish...
6,Barna Saha,Algorithm Design,Not Found,Not Found,Not Found,Not Found,"theoretical computer science, Probability, Geo...",Not Found,Not Found,Not Found,Not Found,https://simons.berkeley.edu/people/barna-saha
7,Gireeja Ranade,Not Found,Scientist.,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,https://simons.berkeley.edu/people/gireeja-ranade
8,Ned Mohan,Not Found,Not Found,Not Found,Not Found,Regents Professor,Not Found,Not Found,Not Found,Not Found,Not Found,https://ece.umn.edu/directory/mohan-ned/
9,Venkat Anantharam,"member of the faculty at Cornell University.,P...",Not Found,Not Found,Not Found,Not Found,"Indian Institute of Technology (1980), an MS i...",Not Found,Not Found,Not Found,Not Found,https://simons.berkeley.edu/people/venkat-anan...


In [ ]:
newdatadf.to_csv('my-drive/My Drive/new_caltech_data.csv',index=False)